In [80]:
pip install sqlite3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [140]:
from pyspark.sql import SparkSession
import os
import sys
from config import API_KEY
from datetime import datetime, timedelta
import requests
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, DateType, DoubleType, IntegerType
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col, rank, avg, round, count, countDistinct, udf
import sqlite3

In [6]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

#os.environ["PYSPARK_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"
#os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"

JAVA_HOME: None
PATH: c:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\usr\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Scripts;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\bin;C:\Users\lucas\anaconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporati

In [3]:
import findspark
findspark.init()

In [20]:
spark.stop()

In [21]:
spark = SparkSession.builder \
    .appName("weatherapp") \
    .getOrCreate()
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
    #.config("spark.python.worker.memory", "4g") \
    #.config("spark.driver.maxResultSize", "4g") \
    #.config("spark.pyspark.python", r"C:\Users\lucas\anaconda3\python.exe") \
    #.config("spark.pyspark.driver.python", r"C:\Users\lucas\anaconda3\python.exe") \
#spark.sparkContext.addFile(r"C:\Users\lucas\anaconda3\python.exe") 

In [ ]:
url = "http://api.weatherapi.com/v1/current.json"
#http://api.weatherapi.com/v1/history.json
api_key = API_KEY
city = 'Philadelphia'
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)
start_date = start_date.strftime("%Y-%m-%d")
params = {
    'key': api_key,
    'q': city
}

response = requests.get(url,params=params)
data = response.json()



In [36]:
data['current']

{'last_updated_epoch': 1732400100,
 'last_updated': '2024-11-23 17:15',
 'temp_c': 10.3,
 'temp_f': 50.5,
 'is_day': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 13.2,
 'wind_kph': 21.2,
 'wind_degree': 292,
 'wind_dir': 'WNW',
 'pressure_mb': 1008.0,
 'pressure_in': 29.75,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 48,
 'cloud': 50,
 'feelslike_c': 7.6,
 'feelslike_f': 45.8,
 'windchill_c': 8.0,
 'windchill_f': 46.4,
 'heatindex_c': 10.7,
 'heatindex_f': 51.2,
 'dewpoint_c': 4.5,
 'dewpoint_f': 40.1,
 'vis_km': 16.0,
 'vis_miles': 9.0,
 'uv': 0.0,
 'gust_mph': 17.9,
 'gust_kph': 28.8}

In [ ]:
#history
def loop_date(start_date,end_date):
    date_list = []
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += timedelta(days=1)
    return date_list

def get_city_attr(cities,start_date,end_date):
    url = "http://api.weatherapi.com/v1/history.json"
    api_key = API_KEY
    date_list = loop_date(start_date,end_date)
    measure_metrics = ['avgtemp_f','maxwind_mph','totalprecip_in','condition']
    spark_values_store = []
    for city in cities:
        for date in date_list:
            params = {
                'key': api_key,
                'q': city,
                'dt': date
            }
            response = requests.get(url,params=params)
            data = response.json()
            date_store = {'date':date, 'city':city}
            for metric in measure_metrics:
                date_store[metric] = data['forecast']['forecastday'][0]['day'][metric]
            spark_values_store.append(date_store)
    return spark_values_store


In [39]:
def get_city_attr(cities,unit ='F'):
    url = "http://api.weatherapi.com/v1/current.json"
    api_key = API_KEY
    measure_metrics = ['name','temp_f','humidity','condition']
    spark_values_store = []
    for city in cities:
        params = {
            'key': api_key,
            'q': city
        }
        response = requests.get(url,params=params)
        data = response.json()
        data_store = {}
        for metric in measure_metrics:
            if metric == 'name':
                data_store[metric] = data['location'][metric]
                continue
            if metric == 'condition':
                data_store[metric] = data['current'][metric]['text']
                continue
            data_store[metric] = data['current'][metric]
        if unit == 'C':
            data_store['temp_c'] = (data_store['temp_f'] - 32) * (5/9)
            del data_store['temp_f']
        spark_values_store.append(data_store)
    return spark_values_store


In [29]:
cities = [
    "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose",
    "London", "Paris", "Berlin", "Madrid", "Rome",
    "Tokyo", "Beijing", "Mumbai", "Sydney", "Cape Town",
    "Dubai", "Singapore", "Seoul", "Moscow", "Rio de Janeiro",
    "Buenos Aires", "Cairo", "Bangkok", "Istanbul", "Toronto",
    "Mexico City", "Jakarta", "Kuala Lumpur", "Lagos", "Nairobi",
    "Johannesburg", "Melbourne", "Lima", "Bogotá", "Santiago",
    "Amsterdam", "Brussels", "Warsaw", "Vienna", "Stockholm",
    "Oslo", "Copenhagen", "Helsinki", "Prague", "Zurich"
]

In [12]:
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)

In [13]:
df_list = get_city_attr(cities,start_date,end_date)
    

In [40]:
weather_df = get_city_attr(cities)

In [15]:
schema = StructType([
    StructField('date', StringType(), True),
    StructField('city', StringType(), True),
    StructField('avgtemp_f', FloatType(), True),
    StructField('maxwind_mph', FloatType(), True),
    StructField('totalprecip_in', FloatType(), True)
])

In [ ]:
def clean_record(record):
    return {
        'date' : str(record['date']),
        'city' : str(record['city']),
        'avgtemp_f' : double(record['avgtemp_f']),
        'maxwind_mph' : double(record['maxwind_mph']),
        'totalprecip_in' : float(record['totalprecip_in'])
    }

In [41]:
weather_df = spark.createDataFrame(weather_df)

In [ ]:
#find the city with the highest temperature
window_spec = Window.partitionBy().orderBy(col('temp_f').desc())
df_with_rank = weather_df.withColumn('rank',rank().over(window_spec))
city_with_highest_temp = df_with_rank.filter(col('rank')==1).select('name').collect()[0][0]
print(city_with_highest_temp)


In [ ]:
#compare temperature of 2 cities
def city_compare(city1,city2,df):
    if df.filter(col('name') == city1).select('temp_f').collect()[0][0] == df.filter(col('name') == city2).select('temp_f').collect()[0][0]:
        print(f'The temperatures of {city1} and {city2} are the same')
    elif df.filter(col('name') == city1).select('temp_f').collect()[0][0] > df.filter(col('name') == city2).select('temp_f').collect()[0][0]:
        diff_temp = df.filter(col('name') == city1).select('temp_f').collect()[0][0] - df.filter(col('name') == city2).select('temp_f').collect()[0][0]
        print(f'The temperature of {city1} is greater than {city2} by {diff_temp} degrees Farenheit')
    else:
        diff_temp = df.filter(col('name') == city2).select('temp_f').collect()[0][0] - df.filter(col('name') == city1).select('temp_f').collect()[0][0]
        print(f'The temperature of {city2} is greater than {city1} by {diff_temp} degrees Farenheit')

In [68]:
city_compare('Singapore','Sydney',weather_df)

The temperature of Singapore is greater than Sydney by 8.400000000000006 degrees Farenheit


In [62]:
df_with_rank.show()

+-------------+--------+--------------+------+----+
|    condition|humidity|          name|temp_f|rank|
+-------------+--------+--------------+------+----+
|         Mist|      63|       Jakarta|  89.8|   1|
|Partly cloudy|      70|     Singapore|  86.5|   2|
|Partly cloudy|      66|       Bangkok|  86.4|   3|
|Partly cloudy|      79|  Kuala Lumpur|  84.6|   4|
|        Clear|      94|         Lagos|  79.3|   5|
|        Sunny|      65|        Sydney|  78.1|   6|
|        Sunny|      69|         Dubai|  77.4|   7|
|Partly cloudy|      65|     Melbourne|  77.0|   8|
|     Overcast|      44|        Mumbai|  75.4|   9|
|     Overcast|      78|  Buenos Aires|  72.0|  10|
|        Clear|      73|Rio De Janeiro|  72.0|  10|
|Partly cloudy|      21|       Phoenix|  71.1|  12|
|Partly Cloudy|      82|          Lima|  65.7|  13|
|        Clear|      47|   San Antonio|  64.9|  14|
|     Overcast|      64|       Houston|  64.4|  15|
|Partly cloudy|      60|     San Diego|  63.0|  16|
|        Cle

In [78]:
def average_weather(cities,df):
    filtered_df = df.filter(col('name').isin(cities))
    avg_temp = filtered_df.agg(avg('temp_f').alias('avg_temp')).collect()[0]['avg_temp']
    avg_humidity = filtered_df.agg(avg('humidity').alias('avg_humidity')).collect()[0]['avg_humidity']
    print(f'The average temp of these cities is {avg_temp} and the average humidity of these cities is {avg_humidity}')

In [72]:
cities_for_temp_analysis = [
    "New York", "Los Angeles", "Tokyo", "Cairo", "Moscow",
    "Mumbai", "Sydney", "Cape Town", "Dubai", "Toronto",
    "Mexico City", "Lagos", "Bangkok", "Istanbul", "Berlin",
    "London", "Buenos Aires", "Jakarta", "Beijing", "Melbourne"
]

In [79]:
average_weather(cities_for_temp_analysis,weather_df)

The average temp of these cities is 61.36500000000001 and the average humidity of these cities is 65.0


In [ ]:
#create sqlite database
conn = sqlite3.connect('weather.db')

In [84]:
#create sqllite database
pandas_df = weather_df.toPandas()
pandas_df.to_sql("weather",conn,if_exists='replace',index=True)

50

In [85]:
#create sqllite database
conn.close()

In [86]:
conn = sqlite3.connect('weather.db')
result = conn.execute('SELECT * FROM weather').fetchall()

In [88]:
conn.close()

In [89]:
weather_df.show()

+-------------+--------+------------+------+
|    condition|humidity|        name|temp_f|
+-------------+--------+------------+------+
|     Overcast|      41|    New York|  53.1|
|Partly cloudy|      68| Los Angeles|  61.2|
|     Overcast|      67|     Chicago|  42.1|
|     Overcast|      64|     Houston|  64.4|
|Partly cloudy|      21|     Phoenix|  71.1|
|     Overcast|      61|Philadelphia|  52.0|
|        Clear|      47| San Antonio|  64.9|
|Partly cloudy|      60|   San Diego|  63.0|
|Partly cloudy|      69|      Dallas|  48.9|
|Partly cloudy|      80|    San Jose|  60.1|
|Partly cloudy|      81|      London|  43.3|
|     Overcast|      93|       Paris|  48.9|
|   Light rain|      87|      Berlin|  48.2|
|        Clear|      87|      Madrid|  41.7|
|Partly cloudy|      94|        Rome|  52.2|
|Partly cloudy|      54|       Tokyo|  53.6|
|        Sunny|      21|     Beijing|  34.5|
|     Overcast|      44|      Mumbai|  75.4|
|        Sunny|      65|      Sydney|  78.1|
|   Light 

In [92]:
agg_df = weather_df.groupby('condition').agg(
    avg('humidity').alias('average_humidity'),
    avg('temp_f').alias('average_temp')
)

In [93]:
agg_df.show()

+-------------+------------------+------------------+
|    condition|  average_humidity|      average_temp|
+-------------+------------------+------------------+
|     Overcast| 69.77777777777777| 54.93333333333333|
|Partly cloudy|              73.0| 58.50555555555555|
|        Clear| 70.45454545454545|57.354545454545466|
|   Light rain| 89.71428571428571| 47.55714285714286|
|        Sunny|51.666666666666664|63.333333333333336|
|         Mist|              63.0|              89.8|
|Partly Cloudy|              82.0|              65.7|
+-------------+------------------+------------------+



In [95]:
weather_df.show()

+-------------+--------+------------+------+
|    condition|humidity|        name|temp_f|
+-------------+--------+------------+------+
|     Overcast|      41|    New York|  53.1|
|Partly cloudy|      68| Los Angeles|  61.2|
|     Overcast|      67|     Chicago|  42.1|
|     Overcast|      64|     Houston|  64.4|
|Partly cloudy|      21|     Phoenix|  71.1|
|     Overcast|      61|Philadelphia|  52.0|
|        Clear|      47| San Antonio|  64.9|
|Partly cloudy|      60|   San Diego|  63.0|
|Partly cloudy|      69|      Dallas|  48.9|
|Partly cloudy|      80|    San Jose|  60.1|
|Partly cloudy|      81|      London|  43.3|
|     Overcast|      93|       Paris|  48.9|
|   Light rain|      87|      Berlin|  48.2|
|        Clear|      87|      Madrid|  41.7|
|Partly cloudy|      94|        Rome|  52.2|
|Partly cloudy|      54|       Tokyo|  53.6|
|        Sunny|      21|     Beijing|  34.5|
|     Overcast|      44|      Mumbai|  75.4|
|        Sunny|      65|      Sydney|  78.1|
|   Light 

In [96]:
high_temp_df = weather_df.filter(weather_df['temp_f'] > 60)

In [97]:
high_temp_df.show()

+-------------+--------+--------------+------+
|    condition|humidity|          name|temp_f|
+-------------+--------+--------------+------+
|Partly cloudy|      68|   Los Angeles|  61.2|
|     Overcast|      64|       Houston|  64.4|
|Partly cloudy|      21|       Phoenix|  71.1|
|        Clear|      47|   San Antonio|  64.9|
|Partly cloudy|      60|     San Diego|  63.0|
|Partly cloudy|      80|      San Jose|  60.1|
|     Overcast|      44|        Mumbai|  75.4|
|        Sunny|      65|        Sydney|  78.1|
|        Sunny|      69|         Dubai|  77.4|
|Partly cloudy|      70|     Singapore|  86.5|
|        Clear|      73|Rio De Janeiro|  72.0|
|     Overcast|      78|  Buenos Aires|  72.0|
|Partly cloudy|      66|       Bangkok|  86.4|
|        Clear|      17|   Mexico City|  62.8|
|         Mist|      63|       Jakarta|  89.8|
|Partly cloudy|      79|  Kuala Lumpur|  84.6|
|        Clear|      94|         Lagos|  79.3|
|Partly cloudy|      94|       Nairobi|  61.3|
|        Clea

In [98]:
weather_df.select('condition','humidity').show()

+-------------+--------+
|    condition|humidity|
+-------------+--------+
|     Overcast|      41|
|Partly cloudy|      68|
|     Overcast|      67|
|     Overcast|      64|
|Partly cloudy|      21|
|     Overcast|      61|
|        Clear|      47|
|Partly cloudy|      60|
|Partly cloudy|      69|
|Partly cloudy|      80|
|Partly cloudy|      81|
|     Overcast|      93|
|   Light rain|      87|
|        Clear|      87|
|Partly cloudy|      94|
|Partly cloudy|      54|
|        Sunny|      21|
|     Overcast|      44|
|        Sunny|      65|
|   Light rain|      82|
+-------------+--------+
only showing top 20 rows



In [101]:
weather_df = weather_df.withColumnRenamed('temp_f','temperature')

In [107]:
weather_df = weather_df.withColumn('temp_c',round((col('temperature')-32) * (5/9),2))

In [ ]:
weather_df.show()

+-------------+--------+------------+-----------+------+
|    condition|humidity|        name|temperature|temp_c|
+-------------+--------+------------+-----------+------+
|     Overcast|      41|    New York|       53.1| 11.72|
|Partly cloudy|      68| Los Angeles|       61.2| 16.22|
|     Overcast|      67|     Chicago|       42.1|  5.61|
|     Overcast|      64|     Houston|       64.4|  18.0|
|Partly cloudy|      21|     Phoenix|       71.1| 21.72|
|     Overcast|      61|Philadelphia|       52.0| 11.11|
|        Clear|      47| San Antonio|       64.9| 18.28|
|Partly cloudy|      60|   San Diego|       63.0| 17.22|
|Partly cloudy|      69|      Dallas|       48.9|  9.39|
|Partly cloudy|      80|    San Jose|       60.1| 15.61|
|Partly cloudy|      81|      London|       43.3|  6.28|
|     Overcast|      93|       Paris|       48.9|  9.39|
|   Light rain|      87|      Berlin|       48.2|   9.0|
|        Clear|      87|      Madrid|       41.7|  5.39|
|Partly cloudy|      94|       

In [113]:
def get_city_attr2(cities,unit ='F'):
    url = "http://api.weatherapi.com/v1/current.json"
    api_key = API_KEY
    measure_metrics = ['name','gust_mph','pressure_mb']
    spark_values_store = []
    for city in cities:
        params = {
            'key': api_key,
            'q': city
        }
        response = requests.get(url,params=params)
        data = response.json()
        data_store = {}
        for metric in measure_metrics:
            if metric == 'name':
                data_store[metric] = data['location'][metric]
                continue
            data_store[metric] = data['current'][metric]
        if unit == 'C':
            data_store['temp_c'] = (data_store['temp_f'] - 32) * (5/9)
            del data_store['temp_f']
        spark_values_store.append(data_store)
    return spark_values_store


In [114]:
weather_df2 = get_city_attr2(cities)

In [115]:
weather_spark_df2 = spark.createDataFrame(weather_df2)

In [116]:
weather_spark_df2.show()

+--------+------------+-----------+
|gust_mph|        name|pressure_mb|
+--------+------------+-----------+
|    17.7|    New York|     1019.0|
|     6.7| Los Angeles|     1020.0|
|    13.0|     Chicago|     1024.0|
|     7.2|     Houston|     1025.0|
|     5.6|     Phoenix|     1020.0|
|    13.4|Philadelphia|     1021.0|
|     5.7| San Antonio|     1025.0|
|     6.7|   San Diego|     1020.0|
|     6.6|      Dallas|     1027.0|
|     2.3|    San Jose|     1021.0|
|    15.5|      London|     1011.0|
|    13.4|       Paris|     1019.0|
|    13.2|      Berlin|     1023.0|
|     4.7|      Madrid|     1025.0|
|     9.4|        Rome|     1019.0|
|    12.4|       Tokyo|     1016.0|
|    17.6|     Beijing|     1018.0|
|    16.5|      Mumbai|     1008.0|
|    11.3|      Sydney|     1007.0|
|    21.5|   Cape Town|     1016.0|
+--------+------------+-----------+
only showing top 20 rows



In [120]:
weather_spark_df3 = weather_df.join(weather_spark_df2, weather_df['name'] == weather_spark_df2['name'], 'inner').select(weather_df.name,weather_df.condition,weather_df.humidity,weather_df.temperature
                                                                                                                        ,weather_df.temp_c,weather_spark_df2.gust_mph,weather_spark_df2.pressure_mb)

In [121]:
weather_spark_df3.show()

+------------+-------------+--------+-----------+------+--------+-----------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|pressure_mb|
+------------+-------------+--------+-----------+------+--------+-----------+
|     Chicago|     Overcast|      67|       42.1|  5.61|    13.0|     1024.0|
|    New York|     Overcast|      41|       53.1| 11.72|    17.7|     1019.0|
| Los Angeles|Partly cloudy|      68|       61.2| 16.22|     6.7|     1020.0|
|     Houston|     Overcast|      64|       64.4|  18.0|     7.2|     1025.0|
|   San Diego|Partly cloudy|      60|       63.0| 17.22|     6.7|     1020.0|
|     Phoenix|Partly cloudy|      21|       71.1| 21.72|     5.6|     1020.0|
|Philadelphia|     Overcast|      61|       52.0| 11.11|    13.4|     1021.0|
| San Antonio|        Clear|      47|       64.9| 18.28|     5.7|     1025.0|
|      London|Partly cloudy|      81|       43.3|  6.28|    15.5|     1011.0|
|      Dallas|Partly cloudy|      69|       48.9|  9.39|     6.6

In [124]:
weather_spark_df3.groupBy('condition').agg(count('*')).show()

+-------------+--------+
|    condition|count(1)|
+-------------+--------+
|     Overcast|       9|
|        Clear|      11|
|   Light rain|       7|
|         Mist|       1|
|Partly Cloudy|       1|
|Partly cloudy|      18|
|        Sunny|       3|
+-------------+--------+



In [125]:
weather_spark_df3 = weather_spark_df3.drop('pressure_mb')

In [127]:
weather_spark_df3 = weather_spark_df3.replace('Light rain','Drizzle','condition')

In [129]:
weather_spark_df3.orderBy('temperature').show()

+----------+-------------+--------+-----------+------+--------+
|      name|    condition|humidity|temperature|temp_c|gust_mph|
+----------+-------------+--------+-----------+------+--------+
|    Moscow|Partly cloudy|      80|       28.4|  -2.0|    16.1|
|    Warsaw|        Clear|      86|       34.2|  1.22|     8.1|
|   Beijing|        Sunny|      21|       34.5|  1.39|    17.6|
|    Vienna|Partly cloudy|      93|       35.6|   2.0|    13.0|
|    Prague|     Overcast|      93|       37.8|  3.22|     5.2|
|    Madrid|        Clear|      87|       41.7|  5.39|     4.7|
|   Chicago|     Overcast|      67|       42.1|  5.61|    13.0|
|    London|Partly cloudy|      81|       43.3|  6.28|    15.5|
|     Seoul|      Drizzle|      93|       43.5|  6.39|     5.3|
|  Helsinki|      Drizzle|      93|       44.6|   7.0|    25.6|
|   Toronto|      Drizzle|      87|       44.8|  7.11|    19.1|
| Amsterdam|Partly cloudy|      87|       44.8|  7.11|    20.0|
|      Oslo|      Drizzle|      93|     

In [138]:
def temp_range(temp):
    if temp < 50:
        return 'Low'
    elif (temp >= 50) & (temp < 70):
        return 'Medium'
    else:
        return 'High'

In [141]:
temp_range_udf = udf(temp_range,StringType())

In [142]:
weather_spark_df3.withColumn('temp_range',temp_range_udf(col('temperature'))).show()

+------------+-------------+--------+-----------+------+--------+----------+
|        name|    condition|humidity|temperature|temp_c|gust_mph|temp_range|
+------------+-------------+--------+-----------+------+--------+----------+
|     Chicago|     Overcast|      67|       42.1|  5.61|    13.0|       Low|
|    New York|     Overcast|      41|       53.1| 11.72|    17.7|    Medium|
| Los Angeles|Partly cloudy|      68|       61.2| 16.22|     6.7|    Medium|
|     Houston|     Overcast|      64|       64.4|  18.0|     7.2|    Medium|
|   San Diego|Partly cloudy|      60|       63.0| 17.22|     6.7|    Medium|
|     Phoenix|Partly cloudy|      21|       71.1| 21.72|     5.6|      High|
|Philadelphia|     Overcast|      61|       52.0| 11.11|    13.4|    Medium|
| San Antonio|        Clear|      47|       64.9| 18.28|     5.7|    Medium|
|      London|Partly cloudy|      81|       43.3|  6.28|    15.5|       Low|
|      Dallas|Partly cloudy|      69|       48.9|  9.39|     6.6|       Low|

In [ ]:
weather_spark_df3.with